## Q1 How many transactions in the dataset?

## Q2 What is the top-3 most expensive item in the database? 
* show the price of item.

## Q3 Show number of customers were born in each month.
* there are 12 months

## Q4 Show top-3 popular items that make most income.
* show the total income of each item.

## Q5 A manager want to crate a birth-month promotion in January 2017.
* How many customers were born in January and also made transcations in January 2017?

## Q6 In January, what were the top-3 favorite items that customer bought with "Bread"?
* show number of times

## Q7 Who are the top-3 customers that pay most money for "Tea"? 
* show total payment of each customer.

## Q8 What are the top-5 items that "Harry Brown" pay most money? 
* show total payment of each item.

## Q9 Who are top-5 customer paid most money in 2016? 
* show total payment of each customer

## Q10 Between January and March 2017, Who are top-3 customers that paid most for Coffee?
* show total payment

In [1]:
import sqlite3
import pandas as pd
con = sqlite3.connect('db.sqlite3')

In [2]:
transaction = pd.read_sql_query("SELECT * FROM myapp_transaction",con,index_col ='id')

In [3]:
df_transaction = pd.read_sql(
    sql="SELECT * FROM myapp_transaction",
    con=con,
    parse_dates={'datetime': {'format': '%Y-%m-%d %H:%M:%S'}},
    index_col ='id'
)

In [4]:
df_transaction.head()

,customer_id,datetime
id,,
1,1,2019-03-10 03:15:17
2,2,2019-03-10 03:16:14
3,1,2019-03-10 03:17:24
4,62,2016-10-30 10:08:41
5,95,2016-10-30 10:13:03


In [5]:
df_customer = pd.read_sql(
    sql="SELECT * FROM myapp_customer",
    con=con,
    parse_dates={'dob': {'format': '%Y-%m-%d'}},
    index_col ='id'
)

In [6]:
df_customer.head()

,firstname,lastname,dob
id,,,
1,David,Smith,1940-04-10
2,Claire,Taylor,1977-06-14
3,Oliver,Evans,1949-11-09
4,Jack,Davies,1949-09-15
5,James,Johnson,1974-06-11


In [7]:
df_record = pd.read_sql(
    sql="SELECT * FROM myapp_record",
    con=con,
    index_col ='id'
)

In [8]:
df_record.head()

,item_id,transaction_id
id,,
1,1,1
2,1,1
3,2,1
4,3,2
5,2,2


In [9]:
df_item = pd.read_sql(
    sql="SELECT * FROM myapp_item",
    con=con,
    index_col ='id'
)
df_item.head()

,name,price
id,,
1,Banana,50
2,Apple,60
3,Cherries,200
4,Bread,340
5,Scandinavian,370


In [10]:
#Q1 How many transactions in the dataset?
df_transaction.count()

customer_id    9531
datetime       9531
dtype: int64

In [11]:
#Q2 What is the top-3 most expensive item in the database?
#show the price of item.
df_item.sort_values(by=['price'],ascending=False).head(3)

,name,price
id,,
32,My-5 Fruit Shoot,400
43,Empanadas,400
7,Jam,400


In [12]:
#Q3 Show number of customers were born in each month.
#there are 12 months
df_customer['month']=df_customer.dob.dt.month
df_customer.groupby(by=['month']).agg({'month':['count']})

,month
,count
month,
1,17
2,7
3,6
4,10
5,6
6,12
7,11
8,12


In [13]:
#Q4 Show top-3 popular items that make most income.¶
#show the total income of each item.
df_record.merge(right=df_item,left_on='item_id',right_index=True)\
    .groupby('name')\
    .agg({'price':['sum']})\
    .sort_values(('price','sum'),ascending=False)\
    .head(3)

,price
,sum
name,
Coffee,2133690
Bread,1130160
Tea,559650


In [14]:
#Q5 A manager want to crate a birth-month promotion in January 2017.
#How many customers were born in January and also made transcations in January 2017?
df1=df_customer[df_customer.month==1]
df_transaction['month_transaction']=df_transaction.datetime.dt.month
df2=df_transaction[df_transaction.month_transaction==1]

df1.merge(
    right=df2,
    left_index=True,
    right_on='customer_id'
).groupby(by=['customer_id'])\
.agg({'customer_id':['count']})\
.count()

customer_id  count    17
dtype: int64

In [38]:
# Q6 In January, what were the top-3 favorite items that customer bought with "Bread"?
#* show number of times


df_all=df_record.merge(right=df_item,left_on='item_id',right_index=True)\
    .merge(right=df_transaction,left_on='transaction_id',right_index=True)

df_all[
        (df_all.datetime.dt.month==1) &\
        (df_all.name=='Bread')
    ]\
    .groupby(['item_id'])\
    .agg({'transaction_id':['count']})\
    .sort_values(('transaction_id','count'),ascending=False)\
.head()

#df_all.head()
# \
#     .groupby(['item_id'])\
#     .agg({'price':['count']})\
#     .sort_values(('price','sum'),ascending=False)\
#     .head(3)

,transaction_id
,count
item_id,
4,552


In [16]:
df_record.merge(right=df_item,left_on='item_id',right_index=True).head()

,item_id,transaction_id,name,price
id,,,,
1,1,1,Banana,50
2,1,1,Banana,50
7,1,3,Banana,50
3,2,1,Apple,60
5,2,2,Apple,60


In [23]:
# Q7 Who are the top-3 customers that pay most money for "Tea"? 
#* show total payment of each customer.
df_all=df_record.merge(right=df_item,left_on='item_id',right_index=True)\
    .merge(right=df_transaction,left_on='transaction_id',right_index=True)\
    .merge(right=df_customer,left_on='customer_id',right_index=True)\

df_all[df_all.name=='Tea']\
    .groupby(['customer_id','firstname','lastname'])\
    .agg({'price':['sum']})\
    .sort_values(('price','sum'),ascending=False)\
    .head(3)

,,,price
,,,sum
customer_id,firstname,lastname,
54,Jacob,Taylor,7410
84,Claire,Wilson,7020
77,Harry,Wilson,7020


In [31]:
#Q8 What are the top-5 items that "Harry Brown" pay most money?
#show total payment of each item.

df_all=df_record.merge(right=df_item,left_on='item_id',right_index=True)\
    .merge(right=df_transaction,left_on='transaction_id',right_index=True)\
    .merge(right=df_customer,left_on='customer_id',right_index=True)\

df_all[(df_all.firstname=='Harry') & (df_all.lastname=='Brown')]\
    .groupby(['item_id','name'])\
    .agg({'price':['sum']})\
    .sort_values(('price','sum'),ascending=False)\
    .head(5)

,,price
,,sum
item_id,name,
10,Coffee,16770
4,Bread,11560
13,Tea,6630
12,Medialuna,2880
6,Hot chocolate,2700


In [33]:
#Q9 Who are top-5 customer paid most money in 2016?
#show total payment of each customer
df_all=df_record.merge(right=df_item,left_on='item_id',right_index=True)\
    .merge(right=df_transaction,left_on='transaction_id',right_index=True)\
    .merge(right=df_customer,left_on='customer_id',right_index=True)\

df_all[df_all.datetime.dt.year==2016]\
    .groupby(['customer_id','firstname','lastname'])\
    .agg({'price':['sum']})\
    .sort_values(('price','sum'),ascending=False)\
    .head(5)

,,,price
,,,sum
customer_id,firstname,lastname,
5,James,Johnson,38130
77,Harry,Wilson,33300
64,George,Smith,33260
25,Jack,Taylor,32540
89,Jack,Evans,32090


In [35]:
#Q10 Between January and March 2017, Who are top-3 customers that paid most for Coffee?¶
#show total payment

df_all=df_record.merge(right=df_item,left_on='item_id',right_index=True)\
    .merge(right=df_transaction,left_on='transaction_id',right_index=True)\
    .merge(right=df_customer,left_on='customer_id',right_index=True)\

df_all[
        (df_all.datetime.dt.year==2017) &\
        (df_all.datetime.dt.month>=1) &\
        (df_all.datetime.dt.month<=3) &\
        (df_all.name=='Coffee')
    ]\
    .groupby(['customer_id','firstname','lastname'])\
    .agg({'price':['sum']})\
    .sort_values(('price','sum'),ascending=False)\
    .head(3)

,,,price
,,,sum
customer_id,firstname,lastname,
36,Thomas,Johnson,16380
18,Bob,Johnson,15600
84,Claire,Wilson,14040
